Created by Joan-Marc Fisa

- Numerai: [FisaGol](https://numer.ai/fisagol)

- Twitter: [@fisagol](https://twitter.com/fisagol)


In [1]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [2]:
!pip install numerapi
!pip install vecstack;

  Created wheel for vecstack: filename=vecstack-0.4.0-py3-none-any.whl size=19878 sha256=731367573ee7fe9cbde3bdf5b2cf7ed4dde3ea6f18d0f503d3dabead9a5c14be
  Stored in directory: /root/.cache/pip/wheels/28/fe/0c/fe8e43660e3316d7ce204e59a79a72246c0ae9b6c5c79841c8
Successfully built vecstack


In [3]:
##################################################################
##################### LIBRARIES ##################################
##################################################################


In [4]:
import os
import gc
import csv
import sys
import glob
import time
from pathlib import Path
from multiprocessing import Pool

import numerapi

import scipy
import numpy as np
import pandas as pd
import tensorflow as tf
import random
import sklearn
from sklearn import (
    feature_extraction, feature_selection, decomposition, linear_model,
    model_selection, metrics, svm, preprocessing, utils
)
from sklearn.preprocessing import StandardScaler,MinMaxScaler, OrdinalEncoder, LabelEncoder,OneHotEncoder
from keras.models import Sequential, model_from_json, load_model
from keras.layers import Dense, Dropout, Activation,LSTM,Bidirectional, MaxPooling2D, Flatten,GRU
from keras.regularizers import l2
from sklearn.model_selection import StratifiedKFold, GroupKFold, GridSearchCV,cross_val_score,KFold, RepeatedStratifiedKFold,train_test_split
from sklearn.metrics import log_loss, make_scorer, mean_squared_error,classification_report,accuracy_score
from keras.utils import np_utils 
from sklearn import preprocessing
from xgboost import XGBRegressor 
from sklearn.cluster import KMeans
import matplotlib as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler,MinMaxScaler

import math
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, BaggingRegressor
from sklearn.linear_model import SGDRegressor
from lightgbm import LGBMRegressor
from sklearn.tree import DecisionTreeRegressor 

import torch.nn as nn
import torch.nn.functional as F
import torch
from vecstack import stacking

from sklearn import metrics

def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred) ** 0.5


In [5]:
##############################################################################
########################## DOWLOAD DATA ######################################
##############################################################################

In [6]:
TOURNAMENT_NAME = "nomi"
TARGET_NAME = f"target"
PREDICTION_NAME = f"prediction"

BENCHMARK = 0
BAND = 0.2

In [7]:
seed = 1
rand = np.random.seed(seed)

In [8]:
napi = numerapi.NumerAPI(verbosity="info")

napi.download_current_dataset(unzip=True)

current_ds = napi.get_current_round()
latest_round = os.path.join('numerai_dataset_'+str(current_ds))

2021-08-28 18:10:32,272 INFO numerapi.utils: starting download
./numerai_dataset_279.zip: 100%|█████████▉| 418M/419M [00:31<00:00, 12.3MB/s]2021-08-28 18:11:04,295 INFO numerapi.base_api: unzipping file...
./numerai_dataset_279.zip: 419MB [00:49, 12.3MB/s]                           

In [9]:
##################################################################
##################### LOAD DATA ##################################
##################################################################

In [10]:
%%time
print("# Loading data...")

training_data = pd.read_csv(os.path.join(latest_round, "numerai_training_data.csv")).set_index("id")
tournament_data = pd.read_csv(os.path.join(latest_round, "numerai_tournament_data.csv")).set_index("id")
validation_data = tournament_data[tournament_data.data_type == "validation"]

print("# All Loaded...")

# Loading data...


./numerai_dataset_279.zip: 419MB [00:54, 7.68MB/s]


# All Loaded...
CPU times: user 1min 8s, sys: 16.1 s, total: 1min 24s
Wall time: 1min 26s


In [11]:
feature_names = [f for f in training_data.columns if f.startswith("feature")]
print(f"Loaded {len(feature_names)} features")

Loaded 310 features


In [12]:
X = training_data[feature_names]
Y = training_data[training_data.columns[-1]]

le = LabelEncoder()
Y_enc = le.fit_transform(Y)

In [13]:
#############################################################################
###########################  CREATING SOME MODELS  ##########################
#############################################################################

In [14]:
%%time

model = GradientBoostingRegressor(random_state=rand)
model.fit(training_data[feature_names], training_data[TARGET_NAME])
gc.collect()



CPU times: user 28min 7s, sys: 897 ms, total: 28min 7s
Wall time: 28min


In [15]:

y_pred_model = model.predict(tournament_data[feature_names])
gc.collect()

tournament_data[PREDICTION_NAME] = y_pred_model

In [16]:
tournament_data[PREDICTION_NAME]

id
n0003aa52cab36c2    0.488127
n000920ed083903f    0.490204
n0038e640522c4a6    0.522209
n004ac94a87dc54b    0.503401
n0052fe97ea0c05f    0.501119
                      ...   
nffd1cda0c61ecdb    0.504454
nffd711b0e6007e3    0.505101
nffd8a4d389df16a    0.498968
nffdec2abd3d4c1e    0.506943
nfff8751e4f86103    0.497827
Name: prediction, Length: 1800934, dtype: float64

In [17]:
##############################################################################
######################### MAKE SUBMISSION #################################### 
##############################################################################

In [18]:
import numerapi
import torch
from torch.nn import Linear
from torch.nn import Sequential
from torch.functional import F

In [19]:
def exposures(x, y):
    x = x - x.mean(dim=0)
    x = x / x.norm(dim=0)
    y = y - y.mean(dim=0)
    y = y / y.norm(dim=0)
    return torch.matmul(x.T, y)

def reduce_exposure(prediction, features, max_exp):
    # linear model of features that will be used to partially neutralize predictions
    lin = Linear(features.shape[1],  1, bias=False)
    lin.weight.data.fill_(0.)
    model = Sequential(lin)
    optimizer = torch.optim.Adamax(model.parameters(), lr=1e-4)
    feats = torch.tensor(np.float32(features)-.5)
    pred = torch.tensor(np.float32(prediction))
    start_exp = exposures(feats, pred[:,None])
    # set target exposure for each feature to be <= current exposure
    # if current exposure is less than max_exp, or <= max_exp if  
    # current exposure is > max_exp
    targ_exp = torch.clamp(start_exp, -max_exp, max_exp)

    for i in range(100000):
        optimizer.zero_grad()
        # calculate feature exposures of current linear neutralization
        exps = exposures(feats, pred[:,None]-model(feats))
        # loss is positive when any exposures exceed their target
        loss = (F.relu(F.relu(exps)-F.relu(targ_exp)) + F.relu(F.relu(-exps)-F.relu(-targ_exp))).sum()
        print(f'       loss: {loss:0.7f}', end='\r')
        if loss < 1e-7:
            neutralizer = [p.detach().numpy() for p in model.parameters()]
            neutralized_pred = pred[:,None]-model(feats)
            break
        loss.backward()
        optimizer.step()
    return neutralized_pred, neutralizer

def reduce_all_exposures(df, column, neutralizers=[],
                                     normalize=True,
                                     gaussianize=True,
                                     era_col="era",
                                     max_exp=0.1):
    unique_eras = df[era_col].unique()
    computed = []
    for u in unique_eras:
        print(u, '\r')
        df_era = df[df[era_col] == u]
        scores = df_era[column].values
        exposure_values = df_era[neutralizers].values
        
        if normalize:
            scores2 = []
            for x in scores.T:
                x = (scipy.stats.rankdata(x, method='ordinal') - .5) / len(x)
                if gaussianize:
                    x = scipy.stats.norm.ppf(x)
                scores2.append(x)
            scores = np.array(scores2)[0]

        scores, neut = reduce_exposure(scores, exposure_values, max_exp)

        scores /= scores.std()

        computed.append(scores.detach().numpy())

    return pd.DataFrame(np.concatenate(computed), columns=column, index=df.index)

In [21]:
tournament_data[PREDICTION_NAME]

id
n0003aa52cab36c2    0.488127
n000920ed083903f    0.490204
n0038e640522c4a6    0.522209
n004ac94a87dc54b    0.503401
n0052fe97ea0c05f    0.501119
                      ...   
nffd1cda0c61ecdb    0.504454
nffd711b0e6007e3    0.505101
nffd8a4d389df16a    0.498968
nffdec2abd3d4c1e    0.506943
nfff8751e4f86103    0.497827
Name: prediction, Length: 1800934, dtype: float64

In [22]:
data_rfe_10 = reduce_all_exposures(tournament_data,
                                   [PREDICTION_NAME],
                                   neutralizers=feature_names,
                                   era_col="era",
                                   max_exp=0.05)

era121 
era122 
era123 
era124 
era125 
era126 
era127 
era128 
era129 
era130 
era131 
era132 
era575 
era576 
era577 
era578 
era579 
era580 
era581 
era582 
era583 
era584 
era585 
era586 
era587 
era588 
era589 
era590 
era591 
era592 
era593 
era594 
era595 
era596 
era597 
era598 
era599 
era600 
era601 
era602 
era603 
era604 
era605 
era606 
era607 
era608 
era609 
era610 
era611 
era612 
era613 
era614 
era615 
era616 
era617 
era618 
era619 
era620 
era621 
era622 
era623 
era624 
era625 
era626 
era627 
era628 
era629 
era630 
era631 
era632 
era633 
era634 
era635 
era636 
era637 
era638 
era639 
era640 
era641 
era642 
era643 
era644 
era645 
era646 
era647 
era648 
era649 
era650 
era651 
era652 
era653 
era654 
era655 
era656 
era657 
era658 
era659 
era660 
era661 
era662 
era663 
era664 
era665 
era666 
era667 
era668 
era669 
era670 
era671 
era672 
era673 
era674 
era675 
era676 
era677 
era678 
era679 
era680 
era681 
era682 
era683 
era684 
era685 
era686 
era687 


KeyboardInterrupt: ignored

In [ ]:
# replace prediction with reduced feature exposure prediction and rescale to [0,1]
tournament_data[PREDICTION_NAME] = data_rfe_10[PREDICTION_NAME]
tournament_data[PREDICTION_NAME] -= tournament_data[PREDICTION_NAME].min()
tournament_data[PREDICTION_NAME] /= tournament_data[PREDICTION_NAME].max()

In [ ]:
tournament_data[PREDICTION_NAME]

In [ ]:
##############################################################################
######################### MAKE SUBMISSION #################################### 
##############################################################################

In [ ]:
tournament_data[PREDICTION_NAME].to_csv('BeeChain_3.csv')

!cp BeeChain_3.csv "drive/My Drive/BeeChain Foundation/"